In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

In [0]:
#dbutils.secrets.listScopes()

In [0]:
#dbutils.secrets.list(scope="databricksscope")

In [0]:
spark.conf.set("fs.azure.account.key.optumsadllssa.dfs.core.windows.net",dbutils.secrets.get(scope="databricksscope",key="adlskey"))

In [0]:
#display(dbutils.fs.ls("abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net"))

In [0]:
data = spark.read.csv('abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/disease.csv',header=True,inferSchema=True)

In [0]:
data.show(5,False)

+---------+----------+------------+
|subgrp_id|disease_id|disease_name|
+---------+----------+------------+
|S101     |110001    |Beriberi    |
|S101     |110002    |Scurvy      |
|S101     |110003    |Goitre      |
|S101     |110004    |Osteoporosis|
|S101     |110005    |Rickets     |
+---------+----------+------------+
only showing top 5 rows



In [0]:
data.count()

60

In [0]:
data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

+---------+----------+------------+
|subgrp_id|disease_id|disease_name|
+---------+----------+------------+
|        0|         0|           0|
+---------+----------+------------+



In [0]:
data.groupby(data.columns).count().where("count>1").show()

+---------+----------+------------+-----+
|subgrp_id|disease_id|disease_name|count|
+---------+----------+------------+-----+
+---------+----------+------------+-----+



In [0]:
data = data.dropDuplicates()

In [0]:

output_container_path = "abfss://optumstagingdata@optumsadllssa.dfs.core.windows.net"
output_blob_folder = "optumstagingdata"
data.coalesce(1).write.format("csv").mode("overwrite").option("header","true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path,"%s/diseasedatastaging.csv"%output_container_path)

True